In [1]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [46]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [47]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [48]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [49]:
data[:2]

[{'reviewerID': 'A238V1XTSK9NFE',
  'asin': 'B00004VX3T',
  'reviewerName': 'Andrew Lynn',
  'helpful': [2, 2],
  'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automa

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [50]:
y = [rating['overall'] for rating in data]
y[:10]

[5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class KeySelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [rating[self.key] for rating in X]


In [9]:
# Test
# k = KeySelector('reviewText')
# X_trans = k.fit_transform(data)
# X_trans[:3]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import Ridge, Lasso, SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

In [11]:
X_train, X_test, y_train, y_test = \
train_test_split(data, y, test_size = 0.75, random_state=22)

In [12]:
params = {'alpha': np.arange(0.5,3.5,0.5)}

gs = GridSearchCV(Ridge(),
                  param_grid=params,
                  cv=3, 
                  verbose=1, 
                  n_jobs=-1)

In [13]:
bag_of_words_model = Pipeline([
    ('selector', KeySelector('reviewText')),
#     ('vectorizer', CountVectorizer()),
    ('vectorizer', HashingVectorizer()),
#     ('regressor', Ridge()),
    ('predictor', gs)
])

bag_of_words_model.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   15.1s finished


Pipeline(memory=None,
         steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer',
                 HashingVectorizer(alternate_sign=True, analyzer='word',
                                   binary=False, decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, n_features=1048576,
                                   ngram_range=(1, 1), norm='l2',
                                   preprocessor=None, stop_words=None,
                                   strip_accents=None...
                 GridSearchCV(cv=3, error_score='raise-deprecating',
                              estimator=Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_stat

In [14]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score:  1.7518202361034354


# Online Learning - Office Hours

In [15]:
X_mbatch1, X_mbatch2, y_mbatch1, y_mbatch2 = \
train_test_split(data, y, test_size = 0.5, random_state=42)

In [16]:
bag_of_words_trans = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer(n_features=5)),
])

In [17]:
sgd = SGDRegressor(warm_start=True, random_state=42)

# Batch 1
X_trans = bag_of_words_trans.fit_transform(X_mbatch1)
sgd.partial_fit(X_trans, y_mbatch1)
print('mini-batch 1 coef: ', sgd.coef_)

# Batch 2
X_trans = bag_of_words_trans.fit_transform(X_mbatch2)
sgd.partial_fit(X_trans, y_mbatch2)
print('mini-batch 2 coef: ', sgd.coef_)

mini-batch 1 coef:  [-0.97481933  4.07047242  1.60647176 -0.06371551  1.11990225]
mini-batch 2 coef:  [-0.89516154  3.56247906  1.43824927 -0.07306978  0.94211615]


In [18]:
# Single batch

sgd2 = SGDRegressor(random_state=42)
X_trans = bag_of_words_trans.fit_transform(data)
sgd2.partial_fit(X_trans, y)
print('Single batch 2 coef: ', sgd2.coef_)

Single batch 2 coef:  [-0.96078461  3.56525359  1.4683755  -0.08363887  1.01424665]


In [19]:
print('online learning (mini-batches):', sgd.score(X_trans, y))
print('batch learning:', sgd2.score(X_trans, y))

online learning (mini-batches): -3.1633476808166554
batch learning: -3.1617595085963544


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [20]:
from sklearn.decomposition import TruncatedSVD

In [21]:
normalized_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer(n_features=100_000)),
#     ('reduce', TruncatedSVD(n_components=10_000)),
#     ('vectorizer', TfidfVectorizer()),
    ('regressor', Ridge())
])

params = {'vectorizer__lowercase': [True,  False],
          'vectorizer__stop_words': [None, 'english'],
#           'vectorizer_norm':['l1', 'l2'],
          'regressor__alpha': np.arange(0.5,3.5,0.5),
#           'reduce__n_components': np.arange(100,1000,50)
         }

gs = GridSearchCV(normalized_model,
                  param_grid=params,
                  cv=5,
                  verbose=1,
                  n_jobs=-1)

# normalized_model.fit(data, y)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  4.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('selector',
                                        KeySelector(key='reviewText')),
                                       ('vectorizer',
                                        HashingVectorizer(alternate_sign=True,
                                                          analyzer='word',
                                                          binary=False,
                                                          decode_error='strict',
                                                          dtype=<class 'numpy.float64'>,
                                                          encoding='utf-8',
                                                          input='content',
                                                          lowercase=True,
                                                          n_features=100000,
                           

In [22]:
grader.score.nlp__normalized_model(gs.predict)

Your score:  0.9458189838314207


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [23]:
bigrams_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer(ngram_range=(1,2))),
    ('regressor', Ridge(alpha=0.6))
    ])

In [32]:
bigrams_model.fit(X_train, y_train);

In [30]:
bigrams_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer()),
    ('regressor', Ridge())
])
params = {'vectorizer__lowercase': [True],
          'vectorizer__stop_words': ['english'],
          'vectorizer__ngram_range':[(1,1)],
          'regressor__alpha': np.arange(0.6)}

gs = GridSearchCV(bigrams_model,
                  param_grid=params,
                  cv=3,
                  verbose=1,
                  n_jobs=-1)

gs.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   45.4s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('selector',
                                        KeySelector(key='reviewText')),
                                       ('vectorizer',
                                        HashingVectorizer(alternate_sign=True,
                                                          analyzer='word',
                                                          binary=False,
                                                          decode_error='strict',
                                                          dtype=<class 'numpy.float64'>,
                                                          encoding='utf-8',
                                                          input='content',
                                                          lowercase=True,
                                                          n_features=1048576,
                          

In [34]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score:  0.9303094309744082


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [35]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



To avoid memory issue, we can delete the older data.

In [36]:
del data, y

In [37]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [row['overall'] for row in data_polarity]

In [38]:
pipe = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words='english')),
    ('predictor', MultinomialNB())
])

pipe.fit(data_polarity, ratings)

Pipeline(memory=None,
         steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('predictor',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=

In [39]:
# got log probability from model
log_prob = pipe['predictor'].feature_log_prob_

# got features from model
feature_to_token = pipe['vectorizer'].get_feature_names()

# collapsed log probability into one row
polarity = log_prob[0,:]-log_prob[1,:]

# combine polarity and feature names
most_polar=sorted(list(zip(polarity, feature_to_token)))

# get top and bottom of most popular
n=25
most_polar = most_polar[:n]+most_polar[-n:]

# Get only terms from most_popular
top_50 = [terms for score,terms in most_polar]

In [40]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [82]:
from sklearn.decomposition import NMF
from sklearn.pipeline import FeatureUnion

union = FeatureUnion([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(max_features=1500, min_df=10, stop_words='english')),
    ('predictor', NMF(n_components=5, solver="mu"))
])

union.fit_transform(data_polarity, ratings);

AttributeError: 'dict' object has no attribute 'lower'

In [72]:
words = np.array(pipe['vectorizer'].get_feature_names())

In [73]:
nmf_topic = pipe['predictor'].components_

In [74]:
for i, topic in enumerate(topic):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))

TypeError: 'numpy.float64' object is not iterable

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*